

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [2]:
import random
import string

import torch
torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])

def s_to_idx(s):
    """ Converts a string to a list of integers """
    return [(ord(c) - ord('a') +1) for c in s]

def idx_to_s(idx):
    """ Converts a list of integers to a string """
    return ''.join([chr(c + ord('a') - 1) for c in idx])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [3]:
import torch
class GRUMemory(torch.nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #TODO: initialize your submodules
        self.hidden_size = hidden_size
        # self.embedding_size = 10
        # self.embedding = torch.nn.Embedding(N+1, self.embedding_size)
        
        
        # do not use embedding here
        self.gru = torch.nn.GRU(N+1,
                                self.hidden_size, 
                                num_layers=1,
                                batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, N+1)
        self.hidden = None

    def forward(self, x):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        embedding = x
        gru_outs, _ = self.gru(embedding)
        logits = self.linear(gru_outs)
        return logits

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z. 
        # You need to map those characters to one-hot encodings, 
        # then get the result from your network, and then convert the output 
        # back to a string of the same length, with 0 mapped to ' ', 
        # and 1-26 mapped to a-z.
        idx = s_to_idx(s)
        idx = torch.tensor(idx, dtype=torch.int64)
        onehot = idx_to_onehot(torch.tensor(idx, dtype=torch.int64)).unsqueeze(0)
        logits = self.forward(onehot)
        pred = torch.argmax(logits, dim=2)
        pred = pred.squeeze(0)
        pred_str = ''.join([chr(c + ord('a') - 1) for c in pred])
        return pred_str
    
    # @torch.no_grad()
    # def test_run_tensor(self, s_tensor):
    #     # This function accepts one string s containing lowercase characters a-z. 
    #     # You need to map those characters to one-hot encodings, 
    #     # then get the result from your network, and then convert the output 
    #     # back to a string of the same length, with 0 mapped to ' ', 
    #     # and 1-26 mapped to a-z.
        
    #     # s_tensor = s_tensor.view(1, len(s_tensor), N+1)
    #     logits = self.forward(s_tensor)
    #     pred = torch.argmax(logits, dim=2)
    #     pred = pred.squeeze(0)
    #     pred_str = ''.join([chr(c + ord('a')) for c in pred])
    #     return pred_str

In [4]:
# test run
model = GRUMemory(hidden_size=10)
model.test_run('hello')

/tmp/ipykernel_619588/257215062.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  onehot = idx_to_onehot(torch.tensor(idx, dtype=torch.int64)).unsqueeze(0)


'swwwc'

In [5]:
model == model.cuda()

True

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [6]:
D = 4
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-D], result[D:]):
            correct += int(c1 == c2)
        total += len(s) - D

    return correct / total

In [7]:
DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

In [19]:
import time

start_time = time.time()


model = GRUMemory(hidden_size=128).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10, gamma=0.1)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[10, 30], gamma=0.1)
criterion = torch.nn.CrossEntropyLoss()
# TODO
epoch = 0
test_acc = 0
while test_acc <= 0.99:
    ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)
    train_dataloader = torch.utils.data.DataLoader(ds, batch_size=512)
    for x, y in train_dataloader:
        batch_size = x.shape[0]
        seq_len = x.shape[1]
        model.train()
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        x_onehot = idx_to_onehot(x)
        y_onehot = idx_to_onehot(y)

        logits = model(x_onehot)
        logits = logits[:, 4:, :]
        y = y[:, 4:]

        loss = criterion(logits.reshape(-1, N + 1), y.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()

    test_acc = test_model(model)
    scheduler.step()
    print(f'epoch {epoch}: {test_acc}', f'loss: {loss.item()}')
    epoch += 1

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

/tmp/ipykernel_605490/257215062.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  onehot = idx_to_onehot(torch.tensor(idx, dtype=torch.int64)).unsqueeze(0)


epoch 0: 0.9997499687460932 loss: 0.0009174463921226561
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [8]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):
    def __init__(self, max_delay=8, seq_length=20, size=1000):
        self.max_delay = max_delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            delay = random.randint(0, self.max_delay)
            result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
            yield seq, delay, result

In [9]:
dataset = VariableDelayEchoDataset(max_delay=8, seq_length=20, size=1000)
for seq, delay, result in dataset:
    print(seq, delay, result)
    break

tensor([25,  3, 20,  3,  3,  9,  2, 24, 13, 20, 19, 25, 12,  8, 26,  1, 13, 25,
        20, 11]) 3 tensor([ 0,  0,  0, 25,  3, 20,  3,  3,  9,  2, 24, 13, 20, 19, 25, 12,  8, 26,
         1, 13])


### Model

And the model.

In [10]:
class VariableDelayGRUMemory(torch.nn.Module):
    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.max_delay = max_delay
        self.delay_embedding = torch.nn.Embedding(max_delay + 1, hidden_size//2)
        self.embedding = torch.nn.Embedding(N + 1, hidden_size//2)
        self.gru = torch.nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, N + 1)
        
    def forward(self, x, delays):
        # inputs:
        # x - tensor of shape (batch size, seq length, N + 1)
        # delays - tensor of shape (batch size)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        # (batch size, seq length, N + 1)
        embedding = self.embedding(x)
        
        
        delay_embedding = self.delay_embedding(delays).unsqueeze(1)
        delay_embedding = delay_embedding.repeat(1, embedding.shape[1], 1)
        
        
        # print(delay_embedding.shape, embedding.shape)
        final_embedding = torch.concat((delay_embedding, embedding), dim=2)
        # final_embedding = embedding + delay_embedding
        output, _ = self.gru(final_embedding)
        logits = self.linear(output)
        return logits

    @torch.no_grad()
    def test_run(self, s, delay):
        # This function accepts one string s containing lowercase characters a-z, 
        # and a delay - the desired output delay.
        # You need to map those characters to one-hot encodings, 
        # then get the result from your network, and then convert the output 
        # back to a string of the same length, with 0 mapped to ' ', 
        # and 1-26 mapped to a-z.

        idx = s_to_idx(s)
        idx = torch.tensor(idx, dtype=torch.int64)
        idx = idx.unsqueeze(0)
        delay = torch.tensor(delay, dtype=torch.int64).unsqueeze(0)
        # idx = idx.view(1, len(idx), N+1)
        logits = self.forward(idx, delay)
        pred = torch.argmax(logits, dim=2)
        pred = pred.squeeze(0)
        pred_str = ''.join([chr(c + ord('a') - 1) for c in pred])
        return pred_str

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [11]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [37]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

model = VariableDelayGRUMemory(hidden_size=512, max_delay=MAX_DELAY).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[100, 200, 300], gamma=0.8)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10, gamma=0.5)
criterion = torch.nn.CrossEntropyLoss(reduction="none")

epoch = 0
test_acc = 0
while test_acc <= 0.992:
    dataset = VariableDelayEchoDataset(max_delay=8, seq_length=20, size=2000)
    train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)
    for x, delay, y in train_dataloader:
        batch_size = x.shape[0]
        seq_len = x.shape[1]
        model.train()
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()

        logits = model(x, delay)
        
        mask = [torch.zeros(d) for d in delay]
        mask.append(torch.zeros(SEQ_LENGTH))
        mask = torch.nn.utils.rnn.pad_sequence(mask, padding_value=1, batch_first=True)[:-1]
        logits = logits.reshape(-1, N + 1)
            
        loss = criterion(logits, y.reshape(-1))
        loss = loss.reshape(batch_size, SEQ_LENGTH)
        loss = torch.mean(loss * mask)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.0005)
        
        optimizer.step()

    test_acc = test_variable_delay_model(model)
    # scheduler.step()
    print(f'epoch {epoch}: {test_acc}', f'loss: {loss.item()}')
    epoch += 1

end_time = time.time()
duration = end_time - start_time
accuracy = test_variable_delay_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')


epoch 0: 0.2729652249781877 loss: 1.6668933629989624
epoch 1: 0.4232469993682881 loss: 1.2804588079452515
epoch 2: 0.50125 loss: 0.8192830085754395
epoch 3: 0.5381036217303823 loss: 0.9128271341323853
epoch 4: 0.5881170707197588 loss: 0.7976838946342468
epoch 5: 0.6471746891542534 loss: 0.7809659242630005
epoch 6: 0.6696793363872725 loss: 0.7855949997901917
epoch 7: 0.6588368295440271 loss: 0.8515223860740662
epoch 8: 0.6766527619769941 loss: 0.7032785415649414
epoch 9: 0.7011335012594458 loss: 0.7776274681091309
epoch 10: 0.7268970698722765 loss: 0.5595321655273438
epoch 11: 0.7382666997765086 loss: 0.587973415851593
epoch 12: 0.7606000495908752 loss: 0.5595741271972656
epoch 13: 0.7769249814310473 loss: 0.5864611268043518
epoch 14: 0.7705368289637953 loss: 0.34679415822029114
epoch 15: 0.7708647561588738 loss: 0.3703606128692627
epoch 16: 0.7685196910219071 loss: 0.3569595515727997
epoch 17: 0.7974191931846655 loss: 0.4658064842224121
epoch 18: 0.7938921704159859 loss: 0.344597429037